In [ ]:
import numpy as np
import math
import CellModeller
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
import pickle
import random
from CellModeller.Regulation.ModuleRegulator import ModuleRegulator
%matplotlib notebook

In [ ]:
#list of cellular properties

data = pd.read_pickle('/Users/travis/Cellmodeller/data/RbrRBe_DegSig_PollRbeIB_IBMaturetoEB063021-21-07-02-17-42/step-00010.pickle')
cs = data['cellStates']

ids = np.array([cell.id for (id,cell) in cs.items()])
lengths = np.array([cell.length for (id,cell) in cs.items()])
pos = np.array([cell.pos for (id,cell) in cs.items()])
ctype = np.array([cell.cellType for (id,cell) in cs.items()])
ccolor = np.array([cell.color for (id,cell) in cs.items()])
cellage = np.array([cell.cellAge for (id,cell) in cs.items()])
cellradius = np.array([cell.radius for (id,cell) in cs.items()])
cellvolume = np.array([cell.volume for (id,cell) in cs.items()])
celldivisions = np.array([cell.cellAdh for (id,cell) in cs.items()])
cellends = np.array([cell.ends for (id,cell) in cs.items()])
celldir = np.array([cell.dir for (id,cell) in cs.items()])
celleffGrowth = np.array([cell.effGrowth for (id,cell) in cs.items()])
cellexcludeAttr = np.array([cell.excludeAttr for (id,cell) in cs.items()])
celldivideFlag = np.array([cell.divideFlag for (id,cell) in cs.items()])
cellgrowthRate = np.array([cell.growthRate for (id,cell) in cs.items()])
cellcts = np.array([cell.cts for (id,cell) in cs.items()])
species = np.array([cell.species for (id,cell) in cs.items()])
geneamt = np.array([cell.geneamt for (id,cell) in cs.items()])

In [ ]:
#imports all pickle files in designated folders and concatonates the data sets from simulations.

df1 =pd.DataFrame(columns=['time', 'simulation','init','RBr','RBe','IB', 'pEB','mEB','AB_RB','AB_IB','species0_channel','species1_channel','species2_channel','species3_channel'])
df_ctype = pd.DataFrame(columns=['time','simulation','ctype','species0_channel','species1_channel','species2_channel','species3_channel'])
for sim in glob.glob('/Users/travis/Cellmodeller/data/IBpaper_TotalRBcurve_CDIStoconCoinFlipRBIB_IBMaturetoEB111121-22-01-04-15*'):
    print('--------------------------')
    df = pd.DataFrame(columns=['time', 'simulation','init','RBr','RBe','IB', 'pEB','mEB','AB_RB','AB_IB'])
    df_species = pd.DataFrame(columns=['time','simulation','ctype','species0','species1','species2','species3'])
    for filename in glob.glob(sim+'/*.pickle'):
        i = 0
        name = filename.split('/')[5]
        name = name.split('_')[3]
        name = (name.split('-')[5] + '_')
        step = filename.split('/')[6]
        step = step.split('-')[1]
        step = step.split('.')[0]
        step = int(step)
        data = pd.read_pickle(filename)
        cs = data['cellStates']
        ctype = np.array([cell.cellType for (id,cell) in cs.items()])
        geneamt = np.array([cell.geneamt for (id,cell) in cs.items()])
        cellvolume = np.array([cell.volume for (id,cell) in cs.items()])
        time = step
        cell_0 = np.count_nonzero(ctype == 0)
        cell_1 = np.count_nonzero(ctype == 1)
        cell_2 = np.count_nonzero(ctype == 2)
        cell_3 = np.count_nonzero(ctype == 3)
        cell_4 = np.count_nonzero(ctype == 4)
        cell_5 = np.count_nonzero(ctype == 5)
        cell_6 = np.count_nonzero(ctype == 6)
        cell_7 = np.count_nonzero(ctype == 7)
        geneamt0 = geneamt[0,0]
        geneamt1 = geneamt[0,1]
        geneamt2 = geneamt[0,2]
        geneamt3 = geneamt[0,3]
        print('------------------------')
        df = df.append({'time':time, 'simulation':name, 'init_channel':cell_0, 'RBr':cell_1, 'RBe':cell_2,'IB':cell_3, 'pEB':cell_4,'mEB':cell_5,'AB_RB':cell_6,'AB_IB':cell_7,'GFP_channel':geneamt0,'euo_channel':geneamt1,'hctA_channel':geneamt2,'hctB_channel':geneamt3} , ignore_index=True)
    df1 = df1.append(df)

In [ ]:
#Sort simulations by time.

df_ssg = df1.sort_values(by=['time'])

In [ ]:
#Adjust simulation time to chlamydial cycle.

df_ssg['time']=((df_ssg['time']/10))

In [ ]:
#Sum RBr and RBe for total RBs.

df_ssg['RBs'] = df_ssg['RBr'] + df_ssg['RBe']

In [ ]:
#Pivot on time and simulation number.

def pivot(in_df, channel):
    in_df_p = in_df.pivot_table(index='time', columns='simulation', values=channel)
    in_df_p['mean'], in_df_p['std'] , in_df_p['total']= in_df_p.mean(axis=1), in_df_p.std(axis=1),in_df_p.sum(axis=1)
    return in_df_p

In [ ]:
#Pivot dataframe

RBr_p = pivot(df_ssg  ,   'RBr')
RBe_p = pivot(df_ssg  ,   'RBe')
RBs_p = pivot(df_ssg  ,   'RBs')
IB__p = pivot(df_ssg  ,   'IB')
pEB_p =  pivot(df_ssg ,   'pEB')
EB__p = pivot(df_ssg  ,   'mEB')
ABR_p =  pivot(df_ssg ,   'AB_RB')
ABI_p = pivot(df_ssg  ,   'AB_IB')
euo_p = pivot(df_ssg  ,   'euo_channel')
hcA_p = pivot(df_ssg  ,   'hctA_channel')
hcB_p = pivot(df_ssg  ,   'hctB_channel')
GFP_p = pivot(df_ssg  ,   'GFP_channel')

In [ ]:
#Graph individual simulation traces by cell type

with plt.style.context('seaborn-white'):
    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(ncols=5, nrows=1)
    (RBr_p + RBe_p + ABR_p).drop(columns=['mean','std','total']).plot(legend=False, ax=ax1)
    (IB__p + pEB_p).drop(columns=['mean','std','total']).plot(legend=False, ax=ax2)
    (EB__p).drop(columns=['mean','total','std']).plot(legend=False, ax=ax3)
    (IB__p + pEB_p).drop(columns=['mean','total','std']).plot(legend=False, ax=ax4)
    (EB__p).drop(columns=['mean','total','std']).plot(legend=False, ax=ax5)
    
    ax1.set_ylim([-2, 80])
    ax2.set_ylim([-4, 130])
    ax3.set_ylim([-20, 400])
    ax4.set_ylim([0, 400])
    ax5.set_ylim([0, 400])
        
    ax1.set_xlabel('HPI')
    ax2.set_xlabel('HPI')
    ax3.set_xlabel('HPI')
    ax4.set_xlabel('HPI')
    ax5.set_xlabel('HPI')
    
    ax1.set_xlim([0, 50])
    ax2.set_xlim([0, 50])
    ax3.set_xlim([0, 50])
    ax4.set_xlim([0, 50])
    ax5.set_xlim([0, 50])
    
    ax1.set_title('RBs', fontsize=11)
    ax2.set_title('IBs', fontsize=11)
    ax3.set_title('EBs', fontsize=11)
    ax4.set_title('IBs', fontsize=11)
    ax5.set_title('EBs', fontsize=11)
    fig.set_size_inches(18, 3)

In [ ]:
#Plot averages of individual simulations 

from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import LogLocator
c = sns.color_palette('Set1',16).as_hex()
c[1]

with plt.style.context('seaborn-white'):
    fig, (ax1,ax2) = plt.subplots(ncols=2)
    
    def plot_sample_1(sample, color, name, style, mstyle, fcolor, i):
        ax1.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)     
        ax1.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)
    
    plot_sample_1(RBr_p +  RBe_p , c[2],  'Total RBs' , '-', '', 'None', 1)
    plot_sample_1(IB__p   + pEB_p        , c[1],  'IB' , '-', '', 'None', 1)
    plot_sample_1(EB__p          , c[3],  'EB' , '-', '', 'None', 1)
    
    def plot_sample_2(sample, color, name, style, mstyle, fcolor, i):
        ax2.plot(sample.index, sample['mean']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)     
        ax2.fill_between(sample.index, sample['mean']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['mean']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)
    
    plot_sample_2(GFP_p              , c[0],  'GFP' , '-', '', 'None', 1)
    plot_sample_2(euo_p              , c[1],  'euo' , '-', '', 'None', 1)
    plot_sample_2(hcA_p              , c[2],  'hcA' , '-', '', 'None', 1)
    plot_sample_2(hcB_p              , c[3],  'hcB' , '-', '', 'None', 1)
    
with plt.style.context('classic'):
    ax1.legend(loc='upper left', fontsize=10)
    ax2.legend(loc='upper left', fontsize=10)
    
    ax1.set_xlim([0, 50])
    ax2.set_xlim([0, 40])
    ax1.set_ylim([-10, 400])
    
    ax1.set_xlabel('HPI')
    ax2.set_xlabel('HPI')
    
    ax1.arrow(30, -10, 0, 700,   color=c[8], lw=2, alpha=1, head_width = 0)
    ax2.arrow(30, -10, 0, 700,   color=c[8], lw=2, alpha=1, head_width = 0)
    
    fig.set_size_inches(10, 5)

In [ ]:
#Create dataframe with all 3 Identifiers (celltype, CELL_ID, and species association (per timepoint per sim))

df_typespec3 = pd.DataFrame(columns=['sim','time','celltype','cell_id','species0','species1','species2','species3'])
for sim in glob.glob('/Users/travis/Cellmodeller/data/IBpaper_TotalRBcurve_CDIPollRbeIB_IBMaturetoEB110521-22-02-21*'):
    print('--------------------------')
    
    df_typespec2 = pd.DataFrame(columns=['sim','time','celltype','cell_id','species0','species1','species2','species3'])
    for filename in glob.glob(sim+'/*.pickle'):
        
        df = pd.DataFrame()
        df_typespec0 = pd.DataFrame()
        df_typespec1 = pd.DataFrame()
        
        name = filename.split('/')[5]
        name = name.split('_')[3]
        name = (name.split('-')[5] + '_')
        step = filename.split('/')[6]
        step = step.split('-')[1]
        step = step.split('.')[0]
        step = int(step)
        
        data = pd.read_pickle(filename)
        cs = data['cellStates']
        cell_id = pd.DataFrame([cell.id for (id,cell) in cs.items()]) 
        cell_id = cell_id.rename(columns={0: "cell_id"})

        
        ctype1 = pd.DataFrame([cell.cellType for (id,cell) in cs.items()])
        ctype1 = ctype1.rename(columns={0: "celltype"})
        
        df_typeid = pd.concat([cell_id ,ctype1], axis=1, sort=False)
    
        cgeneamt = pd.DataFrame([cell.geneamt for (id,cell) in cs.items()])
        cgeneamt = cgeneamt.rename(columns={0: "GFP_channel",1: "euo_channel",2: "hcA_channel",3: "hcB_channel"})
        euo = cgeneamt['euo_channel'].copy()
        hcA = cgeneamt['hcA_channel'].copy()
        hcB = cgeneamt['hcB_channel'].copy()
        GFP = cgeneamt['GFP_channel'].copy()
        df_typespec0 = pd.concat([df_typeid ,cgeneamt], axis=1, sort=False)
        
        name2 = pd.DataFrame().reindex_like(ctype1)
        name2 = name2.rename(columns={"celltype": "sim"})
        name2['sim'] = name
        
        df_typespec1 = pd.concat([df_typespec0, name2], axis=1, sort=False)
        
        time2 = pd.DataFrame().reindex_like(ctype1) 

        time2 = time2.rename(columns={"celltype": "time"})
        time2['time'] = step
        
        df = pd.concat([df_typespec1, time2], axis=1, sort=False)
        df_typespec2 = pd.concat([df_typespec2,df])
    df_typespec3 = pd.concat([df_typespec3,df_typespec2])

In [ ]:
df_typespec3

In [ ]:
#Change cell_id to str.

df_typespec3['cell_id'] = df_typespec3['cell_id'].astype(str)

In [ ]:
#combine simulation and cell_id for truly unique cell_id.

df_typespec3["cell_id"] = df_typespec3["sim"] + df_typespec3["cell_id"]

In [ ]:
#Adjust time to chlamydial cycle.

df_typespec3['time']=((df_typespec3['time']/10))

In [ ]:
#Make new dataframes on specific celltype

celltype0 = df_typespec3[df_typespec3.celltype == 0]
celltype1 = df_typespec3[df_typespec3.celltype == 1]
celltype2 = df_typespec3[df_typespec3.celltype == 2]
celltype3 = df_typespec3[df_typespec3.celltype == 3]
celltype4 = df_typespec3[df_typespec3.celltype == 4]
celltype5 = df_typespec3[df_typespec3.celltype == 5]
celltype6 = df_typespec3[df_typespec3.celltype == 6]
celltype7 = df_typespec3[df_typespec3.celltype == 7]

In [ ]:
#Pivots on cell_id and time based on celltype.

def pivot3(in_df, channel):
    in_df_p = in_df.pivot_table(index='time', columns='cell_id', values=channel)
    in_df_p['mean'], in_df_p['std'] , in_df_p['total']= in_df_p.mean(axis=1), in_df_p.std(axis=1),in_df_p.sum(axis=1)
    return in_df_p

In [ ]:
#pivots on cell_id within celltype and time.

cell_id_celltype0_euo_p = pivot3(celltype0  ,   'euo_channel')
cell_id_celltype1_euo_p = pivot3(celltype1  ,   'euo_channel')
cell_id_celltype2_euo_p = pivot3(celltype2  ,   'euo_channel')
cell_id_celltype3_euo_p = pivot3(celltype3  ,   'euo_channel')
cell_id_celltype4_euo_p = pivot3(celltype4  ,   'euo_channel')
cell_id_celltype5_euo_p = pivot3(celltype5  ,   'euo_channel')
cell_id_celltype6_euo_p = pivot3(celltype6  ,   'euo_channel')
cell_id_celltype7_euo_p = pivot3(celltype7  ,   'euo_channel')

cell_id_celltype0_hcA_p = pivot3(celltype0  ,   'hcA_channel')
cell_id_celltype1_hcA_p = pivot3(celltype1  ,   'hcA_channel')
cell_id_celltype2_hcA_p = pivot3(celltype2  ,   'hcA_channel')
cell_id_celltype3_hcA_p = pivot3(celltype3  ,   'hcA_channel')
cell_id_celltype4_hcA_p = pivot3(celltype4  ,   'hcA_channel')
cell_id_celltype5_hcA_p = pivot3(celltype5  ,   'hcA_channel')
cell_id_celltype6_hcA_p = pivot3(celltype6  ,   'hcA_channel')
cell_id_celltype7_hcA_p = pivot3(celltype7  ,   'hcA_channel')

cell_id_celltype0_hcB_p = pivot3(celltype0  ,   'hcB_channel')
cell_id_celltype1_hcB_p = pivot3(celltype1  ,   'hcB_channel')
cell_id_celltype2_hcB_p = pivot3(celltype2  ,   'hcB_channel')
cell_id_celltype3_hcB_p = pivot3(celltype3  ,   'hcB_channel')
cell_id_celltype4_hcB_p = pivot3(celltype4  ,   'hcB_channel')
cell_id_celltype5_hcB_p = pivot3(celltype5  ,   'hcB_channel')
cell_id_celltype6_hcB_p = pivot3(celltype6  ,   'hcB_channel')
cell_id_celltype7_hcB_p = pivot3(celltype7  ,   'hcB_channel')

cell_id_celltype0_GFP_p = pivot3(celltype0  ,   'GFP_channel')
cell_id_celltype1_GFP_p = pivot3(celltype1  ,   'GFP_channel')
cell_id_celltype2_GFP_p = pivot3(celltype2  ,   'GFP_channel')
cell_id_celltype3_GFP_p = pivot3(celltype3  ,   'GFP_channel')
cell_id_celltype4_GFP_p = pivot3(celltype4  ,   'GFP_channel')
cell_id_celltype5_GFP_p = pivot3(celltype5  ,   'GFP_channel')
cell_id_celltype6_GFP_p = pivot3(celltype6  ,   'GFP_channel')
cell_id_celltype7_GFP_p = pivot3(celltype7  ,   'GFP_channel')

In [ ]:
#Plot protein/GFP amounts per celltype.

with plt.style.context('seaborn-white'):
    fig, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(ncols=5, nrows=1)
    (cell_id_celltype1_euo_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax1)
    (cell_id_celltype2_euo_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax2)
    (cell_id_celltype3_euo_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax3)
    (cell_id_celltype3_hcA_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax4)
    (cell_id_celltype4_hcB_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax5)

    ax1.set_xlim([0, 45])
    ax2.set_xlim([0, 45])
    ax3.set_xlim([0, 45])
    ax4.set_xlim([0, 45])
    ax5.set_xlim([0, 45])
    
    ax1.set_title('RBr_euo', fontsize=11)
    ax2.set_title('RBe_euo', fontsize=11)
    ax3.set_title('IB_euo', fontsize=11)
    ax4.set_title('IB_hctA', fontsize=11)
    ax5.set_title('EB_hctB', fontsize=11)

    fig.set_size_inches(40, 3)

In [ ]:
#Pivots on cell_id only and time (has all celltypes each cell_id ever was).

cell_id_GFP_p = pivot3(df_typespec3  ,   'GFP_channel')
cell_id_euo_p = pivot3(df_typespec3  ,   'euo_channel')
cell_id_hcA_p = pivot3(df_typespec3  ,   'hcA_channel')
cell_id_hcB_p = pivot3(df_typespec3  ,   'hcB_channel')

In [ ]:
#Plot protein/GFP amounts based on individual cell_id.

with plt.style.context('seaborn-white'):
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(ncols=4, nrows=1)
    (cell_id_euo_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax1)
    (cell_id_hcA_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax2)
    (cell_id_hcB_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax3)
    (cell_id_GFP_p).drop(columns=['total','std','mean']).plot(legend=False, ax=ax4)

    
    ax1.set_xlim([0, 40])
    ax2.set_xlim([0, 40])
    ax3.set_xlim([0, 40])
    ax4.set_xlim([0, 40])
    

    ax1.set_title('euo', fontsize=11)
    ax2.set_title('hctA', fontsize=11)
    ax3.set_title('hctB', fontsize=11)
    ax4.set_title('GFP', fontsize=11)
    
    fig.set_size_inches(20, 3)

In [ ]:
#Plot averages of individual traces.

with plt.style.context('seaborn-white'):
    fig, (ax1) = plt.subplots(ncols=1)
    
    def plot_sample_1(sample, color, name, style, mstyle, fcolor, i):
        ax1.plot(sample.index, sample['total']*i, color, label=name, linestyle = style, marker=mstyle, alpha=0.8, ms = 5, markerfacecolor=fcolor, markeredgecolor=color, markeredgewidth=1)     
        ax1.fill_between(sample.index, sample['total']*i-(sample['std']*i)/math.sqrt(len(sample.columns)), sample['total']*i+(sample['std']*i)/math.sqrt(len(sample.columns)), color=color, alpha=0.2)
    
    plot_sample_1(cell_id_GFP_p  , c[0],  'GFP'   , '-', '', 'None', 1)

    
with plt.style.context('classic'):
    ax1.legend(loc='upper left', fontsize=8)
    ax2.legend(loc='upper left', fontsize=8)

    ax1.set_xlim([0, 50])
    ax1.set_ylim([-100, 4000])
    
    ax1.set_title('hctA$prom$-GFP', fontsize=11)
    
    ax1.arrow(30, -100, 0, 10000,   color=c[8], lw=2, alpha=1, head_width = 0)
    fig.set_size_inches(5, 5)